## Initializations
Should only need to do these once.

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

## Create a faculty candidate population
Here is a table that I've collected to motivate the distribution of areas that I'm using.

| Abbrev. | Field | # of PhDs (%) | # Faculty at ND (%) |
|------ | -------- | ----------- | --------------------- | 
| ast |Astrophysics|   166 (16%) | 8 (19%) |
| cmp | Condensed Matter Physics | 388 (38%) | 7 (17%) |
| hep | High-Energy Physics | 253 (25%) | 12 (28%) |
| net | Network Science (and/or Biophysics if you like) | 105 (10%) | 2 (5%) |
| nuc | Nuclear | 109 (11%) | 13 (31%) |
| -   | Total   | 1021 | 42 |

The "# of PhDs" column comes from an [AIP survey](https://www.aip.org/statistics/data-graphics/number-physics-phds-granted-subfield-physics-departments-classes-2010-2011) using data on graduates from 2010 and 2011.  I've only counted areas that we have in our department, and I've neglected AMO.  I'll use these numbers to sample candidates from.  The "# of Faculty at ND" column comes from counting up people in our department, including the 2020 retirements and also the recent HEP hire.  I'll use these numbers to set the fields in which we search (i.e. retirements proportional to group size).

For the ratio of men to women, I'm just using the roughly [AIP statistics](https://www.aip.org/statistics/data-graphics/percent-phds-earned-women-selected-fields-classes-1981-through-2016) taken from an eyeball average of the most recent data points.

In [2]:
from faculty_hiring.models import CandidatePopulation

fields = {'ast':0.16, 'cmp':0.38, 'hep':0.25, 'net':0.10, 'nuc':0.11, }

pop = CandidatePopulation()
pop.add_attribute('field',fields)
pop.add_attribute('gender',{'M':0.8, 'F':0.2})

## Generate a few random faculty candidates

In [3]:
print("Candidate A:")
print(pop.generate_candidate())

print("Candidate B:")
print(pop.generate_candidate())

print("Candidate C:")
print(pop.generate_candidate())

Candidate A:
Candidate(field='cmp', gender='M', quality=2.2291828133809344)
Candidate B:
Candidate(field='hep', gender='M', quality=2.771147205557464)
Candidate C:
Candidate(field='ast', gender='M', quality=2.309957350576302)


## Generate a whole population of candidates
Note: This method applies Poisson fluctuations to the size of the candidate pool when generating random candidates.

In [4]:
print("A list of candidates:")
p = pop.generate_population(100)
print("Size of the candidate pool: {}".format(p.quality.size))
for f in ['ast','cmp','hep','net','nuc',]:
    mask = (p.field == f)
    gender_mask = p.gender[mask]
    quality_mask = p.quality[mask]
    best_ind = np.argmax(quality_mask)
    print("Number of {} candidates: {}; Best candidate: {} {:.2f}".format(f.upper(),np.count_nonzero(mask),gender_mask[best_ind],quality_mask[best_ind]))

best_ind = np.argmax(p.quality)
print("Top overall candidate: Gender: {}, Field: {}".format(p.gender[best_ind], p.field[best_ind].upper()))

A list of candidates:
Size of the candidate pool: 110
Number of AST candidates: 15; Best candidate: F 2.49
Number of CMP candidates: 32; Best candidate: M 3.37
Number of HEP candidates: 32; Best candidate: M 3.26
Number of NET candidates: 16; Best candidate: M 3.84
Number of NUC candidates: 15; Best candidate: M 3.03
Top overall candidate: Gender: M, Field: NET


## Possible Search Strategies
There are a few possible search strategies that we can consider:
1. Targeted search in one area, pick the best one
1. Open search in all areas, pick the best one
1. Targeted search in one area, pick the best woman if she is not more than 0.25 sigma below the best man
1. Open search in all areas, pick the best woman if she is not more than 0.25 sigma below the best man

So, an *unbiased* approach would need to pick a woman as often as the fraction in the input population.  That would not promote equity (50% women).  An effective strategy would pick a woman 50% of the time.  Relevant metrics to test to evaluate the strategy include fraction of women hired and average quality of candidates.

In [9]:
from faculty_hiring import strategy
from collections import Counter

search_fields = {'ast':0.19, 'cmp':0.17, 'hep':0.28, 'net':0.05, 'nuc':0.31, }

# Experiment parameters
n_trials = 1000
n_cands = 300
tolerance = 0.25

#Initialize some counters
counter_open = Counter()
counter_targeted = Counter()
counter_open_pref = Counter()
counter_targeted_pref = Counter()
quality_open = 0
quality_targeted = 0
quality_open_pref = 0
quality_targeted_pref = 0

for i in range(n_trials):
    
    p = pop.generate_population(100)

    # Pick the field in which we'll be searching
    field = np.random.choice(list(search_fields.keys()),p=list(search_fields.values()))
    
    # Strategy 1: Pick the best in a randomly chosen target field
    best_targeted = strategy.pick_best(pop.record_type, p, {'field':field}, threshold=3.0)
    if best_targeted != None:
        counter_targeted[best_targeted.gender]+=1
        quality_targeted += best_targeted.quality
    
    # Strategy 2: Just pick the best in all fields
    best_open = strategy.pick_best(pop.record_type, p, threshold=3.0)
    if best_open != None:
        counter_open[best_open.gender]+=1
        quality_open += best_open.quality
    
    # Strategy 3: Prefer women in the same randomly chosen target field
    pref_targeted = strategy.pick_pref(pop.record_type, p, {'gender':'F'}, tolerance, filter_criteria={'field':field}, threshold=3.0)
    if pref_targeted != None:
        counter_targeted_pref[pref_targeted.gender]+=1
        quality_targeted_pref += pref_targeted.quality
    
    # Strategy 4: Just pick the best in all fields
    pref_open = strategy.pick_pref(pop.record_type, p, {'gender':'F'}, tolerance, threshold=3.0)
    if pref_open != None:
        counter_open_pref[pref_open.gender]+=1
        quality_open_pref += pref_open.quality

# Trials done, report results:
print('Hire the "best:"')
successes = sum(counter_targeted.values())
print('Targeted search: Fraction of women hired: {:.2f}%, '.format(100*counter_targeted['F']/successes)+
      'Average quality: {:.2f}, '.format(quality_targeted/successes)+
      'Successful Searchers: {:.2f}%'.format(100*successes/n_trials))
successes = sum(counter_open.values())
print('Open search: Fraction of women hired: {:.2f}%, '.format(100*counter_open['F']/successes)+
      'Average quality: {:.2f}, '.format(quality_open/successes)+
      'Successful Searchers: {:.2f}%'.format(100*successes/n_trials))
print('Preferring Women:')
successes = sum(counter_targeted_pref.values())
print('Targeted search: Fraction of women hired: {:.2f}%, '.format(100*counter_targeted_pref['F']/successes)+
      'Average quality: {:.2f}, '.format(quality_targeted_pref/successes)+
      'Successful Searchers: {:.2f}%'.format(100*successes/n_trials))
successes = sum(counter_open_pref.values())
print('Open search: Fraction of women hired: {:.2f}%, '.format(100*counter_open_pref['F']/successes)+
      'Average quality: {:.2f}, '.format(quality_open_pref/successes)+
      'Successful Searchers: {:.2f}%'.format(100*successes/n_trials))

Hire the "best:"
Targeted search: Fraction of women hired: 22.40%, Average quality: 3.39, Successful Searchers: 67.40%
Open search: Fraction of women hired: 21.71%, Average quality: 3.65, Successful Searchers: 99.50%
Preferring Women:
Targeted search: Fraction of women hired: 25.96%, Average quality: 3.38, Successful Searchers: 67.40%
Open search: Fraction of women hired: 38.39%, Average quality: 3.63, Successful Searchers: 99.50%


## Repeat with Slightly Different Assumptions
Since the results depend on how much of a difference we allow between the best female and best male candidate to still consider them equivalent, here's another run but this time with 0.5 sigma instead of 0.25 sigma as the tolerance.

In [10]:
from faculty_hiring import strategy
from collections import Counter

search_fields = {'ast':0.19, 'cmp':0.17, 'hep':0.28, 'net':0.05, 'nuc':0.31, }

# Experiment parameters
n_trials = 1000
n_cands = 300
tolerance = 0.5

#Initialize some counters
counter_open = Counter()
counter_targeted = Counter()
counter_open_pref = Counter()
counter_targeted_pref = Counter()
quality_open = 0
quality_targeted = 0
quality_open_pref = 0
quality_targeted_pref = 0

for i in range(n_trials):
    
    p = pop.generate_population(100)

    # Pick the field in which we'll be searching
    field = np.random.choice(list(search_fields.keys()),p=list(search_fields.values()))
    
    # Strategy 1: Pick the best in a randomly chosen target field
    best_targeted = strategy.pick_best(pop.record_type, p, {'field':field}, threshold=3.0)
    if best_targeted != None:
        counter_targeted[best_targeted.gender]+=1
        quality_targeted += best_targeted.quality
    
    # Strategy 2: Just pick the best in all fields
    best_open = strategy.pick_best(pop.record_type, p, threshold=3.0)
    if best_open != None:
        counter_open[best_open.gender]+=1
        quality_open += best_open.quality
    
    # Strategy 3: Prefer women in the same randomly chosen target field
    pref_targeted = strategy.pick_pref(pop.record_type, p, {'gender':'F'}, tolerance, filter_criteria={'field':field}, threshold=3.0)
    if pref_targeted != None:
        counter_targeted_pref[pref_targeted.gender]+=1
        quality_targeted_pref += pref_targeted.quality
    
    # Strategy 4: Just pick the best in all fields
    pref_open = strategy.pick_pref(pop.record_type, p, {'gender':'F'}, tolerance, threshold=3.0)
    if pref_open != None:
        counter_open_pref[pref_open.gender]+=1
        quality_open_pref += pref_open.quality

# Trials done, report results:
print('Hire the "best:"')
successes = sum(counter_targeted.values())
print('Targeted search: Fraction of women hired: {:.2f}%, '.format(100*counter_targeted['F']/successes)+
      'Average quality: {:.2f}, '.format(quality_targeted/successes)+
      'Successful Searchers: {:.2f}%'.format(100*successes/n_trials))
successes = sum(counter_open.values())
print('Open search: Fraction of women hired: {:.2f}%, '.format(100*counter_open['F']/successes)+
      'Average quality: {:.2f}, '.format(quality_open/successes)+
      'Successful Searchers: {:.2f}%'.format(100*successes/n_trials))
print('Preferring Women:')
successes = sum(counter_targeted_pref.values())
print('Targeted search: Fraction of women hired: {:.2f}%, '.format(100*counter_targeted_pref['F']/successes)+
      'Average quality: {:.2f}, '.format(quality_targeted_pref/successes)+
      'Successful Searchers: {:.2f}%'.format(100*successes/n_trials))
successes = sum(counter_open_pref.values())
print('Open search: Fraction of women hired: {:.2f}%, '.format(100*counter_open_pref['F']/successes)+
      'Average quality: {:.2f}, '.format(quality_open_pref/successes)+
      'Successful Searchers: {:.2f}%'.format(100*successes/n_trials))

Hire the "best:"
Targeted search: Fraction of women hired: 19.18%, Average quality: 3.39, Successful Searchers: 65.70%
Open search: Fraction of women hired: 18.17%, Average quality: 3.66, Successful Searchers: 99.60%
Preferring Women:
Targeted search: Fraction of women hired: 28.77%, Average quality: 3.38, Successful Searchers: 65.70%
Open search: Fraction of women hired: 54.62%, Average quality: 3.56, Successful Searchers: 99.60%


## Conclusions

There are a number of things one could take away from here:
1. If you don't do anything special in trying to hire women, then the fraction of the women you hire will be about the same as what you see in your input population.
1. An open search does nothing, by itself to increase diversity.
1. Hiring women preferentially reduces the overall quality of the candidates you hire, although the effect is small if you institute a threshold for successful hiring.
1. Performing an open search increases the quality of candidates you hire.
1. The above two affects are just manifestations of the same general principle: searching for the best in a subset, on average, yields lower quality than searching for the best in the full set.
1. **Performing an open search, but preferring women candidates yields *better* quality than performing a targeted search without preferring women.**  Basically, we're just canceling the two affects noted separately above.
1. Targeted searches fail to identify a suitably high quality candidate more often than open searches.

There is certainly weak points to this study.
- My data for setting the distribution of candidates is a bit ad-hoc.  I've picked the size of the applicant pool out of thin air.
- I have assumed that every search yields a candidate of suitable quality.  I haven't populated the candidate pool with any candidates below threshold.
- I have assumed that when we rank candidates, we can measure their quality with no error.
- I have assumed that we measure the quality of candidates without any bias either from gender or from the field of study.
- I have based hiring on the current size of the group without regard for age profile or strategic goals.  For example, I havent accounted for Forro or the push to hire in material science/condensed matter.

To tackle the last three, I would probably need a model of the department composition and retirement progression as well as a model of voting, including potentially individual biases of faculty per area.  If I did this, it would also let me explore different voting methodologies, so I really want to.  However, it will take some doing, so I'm going to pause here with this result.